In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import pysal as ps
from shapely.geometry import Point, LineString, Polygon

In [2]:
raw = pd.read_csv('weekly311_0603.csv')
raw.head()

c:\users\owner\anaconda3\envs\thom\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,39303605,5/25/2018 8:23,5/27/2018 0:00,DOB,Department of Buildings,General Construction/Plumbing,Egress - Doors Locked/Blocked/Improper/No Seco...,NaN,11233,2184 FULTON STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.678195,-73.909425,"(40.67819468692885, -73.90942471479995)"
1,39298402,5/26/2018 17:47,5/27/2018 0:00,DOB,Department of Buildings,General Construction/Plumbing,Egress - Doors Locked/Blocked/Improper/No Seco...,NaN,10010,920 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.739691,-73.989423,"(40.739690585179844, -73.98942319221624)"
2,39296273,5/26/2018 16:04,5/27/2018 0:00,DEP,Department of Environmental Protection,Water System,Leak (Use Comments) (WA2),NaN,11209,12 79 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.631126,-74.038225,"(40.63112628027165, -74.03822496740857)"
3,39295626,5/26/2018 11:34,5/27/2018 0:00,DOB,Department of Buildings,General Construction/Plumbing,Egress - Doors Locked/Blocked/Improper/No Seco...,NaN,10458,2485 ELM PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.861382,-73.894103,"(40.86138205958547, -73.89410343591835)"
4,39295554,5/26/2018 12:21,5/27/2018 0:00,DOB,Department of Buildings,General Construction/Plumbing,Site Conditions Endangering Workers,NaN,10457,2185 VALENTINE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.854777,-73.899034,"(40.85477723678787, -73.89903375851317)"


**Counting values in the "Complaint Type" column to determine the top complaints city-wide.**

In [4]:
city_top = raw['Complaint Type'].value_counts()
city_top.head(5)

Noise - Residential                    5181
Request Large Bulky Item Collection    4757
Illegal Parking                        3261
Noise - Street/Sidewalk                2483
Blocked Driveway                       2464
Name: Complaint Type, dtype: int64

**Grouping by Borough to get Top Complaint Types**

In [5]:
grouped = raw.groupby('Borough')['Complaint Type'].value_counts()

In [12]:
bronx = grouped['BRONX']
bronx.head(5)

Complaint Type
Noise - Residential        1077
Noise - Street/Sidewalk     606
Blocked Driveway            500
UNSANITARY CONDITION        381
Illegal Parking             368
Name: Complaint Type, dtype: int64

In [14]:
brooklyn = grouped['BROOKLYN']
brooklyn.head(5)

Complaint Type
Request Large Bulky Item Collection    1835
Noise - Residential                    1617
Illegal Parking                        1173
Blocked Driveway                        845
Noise - Street/Sidewalk                 734
Name: Complaint Type, dtype: int64

In [15]:
manhattan = grouped['MANHATTAN']
manhattan.head(5)

Complaint Type
Noise - Residential                    923
Noise - Street/Sidewalk                888
Request Large Bulky Item Collection    787
Noise                                  476
Illegal Parking                        470
Name: Complaint Type, dtype: int64

In [16]:
queens = grouped['QUEENS']
queens.head(5)

Complaint Type
Request Large Bulky Item Collection    1542
Noise - Residential                    1313
Illegal Parking                        1052
Blocked Driveway                        972
Street Condition                        747
Name: Complaint Type, dtype: int64

In [17]:
si = grouped['STATEN ISLAND']
si.head(5)

Complaint Type
Request Large Bulky Item Collection    346
Noise - Residential                    247
Electronics Waste                      244
Street Condition                       228
Illegal Parking                        192
Name: Complaint Type, dtype: int64

**Grouping by Borough to get Number of Calls**

In [30]:
group_totals = raw.groupby('Borough')['Unique Key'].count()
group_totals

Borough
BRONX             7528
BROOKLYN         15035
MANHATTAN         8854
QUEENS           12579
STATEN ISLAND     2764
Unspecified        750
Name: Unique Key, dtype: int64

**Mapping**

In [94]:
use_raw = raw[['Complaint Type','Latitude','Longitude']]
use_raw = use_raw.dropna(how='any')
geometry = [Point(xy) for xy in zip(use_raw.Longitude, use_raw.Latitude)]
crs = {'init': 'epsg:4326'}
geo_call = GeoDataFrame(use_raw, crs=crs, geometry=geometry)
geo_call.head()

,Complaint Type,Latitude,Longitude,geometry
0,General Construction/Plumbing,40.678195,-73.909425,POINT (-73.90942471 40.67819469)
1,General Construction/Plumbing,40.739691,-73.989423,POINT (-73.98942319 40.73969059)
2,Water System,40.631126,-74.038225,POINT (-74.03822497 40.63112628)
3,General Construction/Plumbing,40.861382,-73.894103,POINT (-73.89410343999999 40.86138206)
4,General Construction/Plumbing,40.854777,-73.899034,POINT (-73.89903375999999 40.85477724)


In [95]:
nta = gpd.read_file('nynta.shp')
nta = nta.to_crs({'init': 'epsg:4326'})
nta.head()

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK88,Borough Park,39247.228028,5.400502e+07,POLYGON ((-73.97604935657381 40.63127590564677...
1,4,Queens,081,QN52,East Flushing,25843.364936,2.945438e+07,POLYGON ((-73.79493246233996 40.75779803007454...
2,4,Queens,081,QN48,Auburndale,32446.878673,3.416422e+07,POLYGON ((-73.77573836927098 40.74332564716398...
3,4,Queens,081,QN51,Murray Hill,33266.904811,5.248828e+07,POLYGON ((-73.80379022888246 40.77561011179249...
4,4,Queens,081,QN27,East Elmhurst,19816.712323,1.972685e+07,POLYGON ((-73.86109724335759 40.76366447708688...


In [96]:
match = gpd.sjoin(geo_call, nta, how='inner', op='intersects')
match.head()

,Complaint Type,Latitude,Longitude,geometry,index_right,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area
0,General Construction/Plumbing,40.678195,-73.909425,POINT (-73.90942471 40.67819469),34,3,Brooklyn,047,BK79,Ocean Hill,22494.582094,2.008991e+07
221,Request Large Bulky Item Collection,40.682369,-73.912401,POINT (-73.91240079000001 40.68236905),34,3,Brooklyn,047,BK79,Ocean Hill,22494.582094,2.008991e+07
267,Request Large Bulky Item Collection,40.681214,-73.916841,POINT (-73.9168406 40.68121405),34,3,Brooklyn,047,BK79,Ocean Hill,22494.582094,2.008991e+07
291,Request Large Bulky Item Collection,40.672697,-73.920608,POINT (-73.92060761 40.67269693),34,3,Brooklyn,047,BK79,Ocean Hill,22494.582094,2.008991e+07
354,Request Large Bulky Item Collection,40.682058,-73.915105,POINT (-73.91510531 40.68205817),34,3,Brooklyn,047,BK79,Ocean Hill,22494.582094,2.008991e+07


In [108]:
types = match.groupby('NTACode')['Complaint Type'].value_counts()

In [118]:
test = types['BK78'].head(1)

In [111]:
nta_list = list(set(match['NTACode']))

In [113]:
top_dict = {}
for nta in nta_list:
    top_dict[nta] = types[nta].head(1)
top_dict

{'BK78': Complaint Type
 Noise - Residential    89
 Name: Complaint Type, dtype: int64, 'MN35': Complaint Type
 Noise - Residential    127
 Name: Complaint Type, dtype: int64, 'BK77': Complaint Type
 Noise - Residential    77
 Name: Complaint Type, dtype: int64, 'QN54': Complaint Type
 Noise - Residential    89
 Name: Complaint Type, dtype: int64, 'MN01': Complaint Type
 Noise - Street/Sidewalk    83
 Name: Complaint Type, dtype: int64, 'BK09': Complaint Type
 Request Large Bulky Item Collection    15
 Name: Complaint Type, dtype: int64, 'QN02': Complaint Type
 Noise - Residential    11
 Name: Complaint Type, dtype: int64, 'BK23': Complaint Type
 Noise - Residential    2
 Name: Complaint Type, dtype: int64, 'MN04': Complaint Type
 Noise - Residential    72
 Name: Complaint Type, dtype: int64, 'BX13': Complaint Type
 Illegal Parking    12
 Name: Complaint Type, dtype: int64, 'BX99': Complaint Type
 Street Condition    6
 Name: Complaint Type, dtype: int64, 'BK19': Complaint Type
 Noise 

In [124]:
import csv

In [126]:
w = csv.writer(open("tops.csv", "w"))
for key, val in top_dict.items():
    w.writerow([key, val])